In [ ]:
import yfinance as yf

from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import os

# Replace 'YOUR_API_KEY' with your Alpha Vantage API key
api_key = os.environ['ALPHA_VANTAGE_API_KEY']
ticker_symbol = 'IBM'

# Fetch historical data for the past 5 years
ts = TimeSeries(key=api_key, output_format='pandas')
df, meta_data = ts.get_daily(symbol=ticker_symbol, outputsize='full')

# Display the historical data
print(df.head())

In [5]:
class Stock:
    def __init__(self, ticker) -> None:
        self.ticker = yf.Ticker(ticker)
        self.info = self.ticker.info
        # self.market_cap = self.info['marketCap']
        # self.quarterly_income_stmt = self.ticker.quarterly_income_stmt
        # self.quarterly_balance_sheet = self.ticker.quarterly_balance_sheet
        # self.quarterly_cash_flow = self.ticker.quarterly_cash_flow

### (1) Market Cap at or below Net-Current-Asset-Value

In [ ]:
msft = Stock("MSFT")

# msft.market_cap
total_assets = msft.quarterly_balance_sheet.loc['Total Assets'][0]
total_liabilities =msft.quarterly_balance_sheet.loc['Current Liabilities'][0]

total_assets-total_liabilities

### (2) Average 5Y annual Free Cash Flow <= 10%

In [ ]:
ba = Stock("BA")

ba.ticker.get_cash_flow(freq='quarterly')

### (3) Positive TTM Free-cashflow

In [ ]:

cn = Stock("CNI")
operating_cash_flow_df = cn.ticker.quarterly_cash_flow.loc['Operating Cash Flow']
capital_expendatures_df = cn.ticker.quarterly_cash_flow.loc['Capital Expenditure']
df['TTM']

### (4) Zero or negative net-debt

In [ ]:
rbc = yf.Ticker("RY.TO")
rbc.quarterly_balance_sheet.head(2)

### (5) Sum 5Y dividends or buybacks present

In [8]:
jpm = yf.Ticker("JPM")
dividend_df = jpm.history(period="5y")["Dividends"]

dividends = dividend_df[dividend_df > 0].to_list()
dividends_paid = (dividend_df > 0).sum()
dividends

[0.8,
 0.8,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.05,
 1.05]

### (6) Only companies from selected country HQs or exchanges

In [9]:
jpm.info

{'regularMarketPrice': None, 'preMarketPrice': None, 'logo_url': ''}

In [1]:
import requests

cik = "0001840574"
url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
OperatingExpenses = f"https://data.sec.gov/api/xbrl/frames/us-gaap/OperatingExpenses/USD/CY2022.json"
NetCashProvidedByUsedInOperatingActivities = f"https://data.sec.gov/api/xbrl/frames/us-gaap/NetCashProvidedByUsedInOperatingActivities/USD/CY2022.json"
req = requests.get(url, headers={"User-Agent":"rie@gmail.com", "Accept-Encoding":"gzip, deflate, br", "Host":"data.sec.gov", "Connection":"keep-alive"})
facts = req.json()['facts']
## FCF = Operating Revenue - Capital Expenditures

In [3]:
def convert_cik_list(file_path: str) -> dict:
    result_dict = {}
    
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split(':')
            if len(parts) == 2:
                ticker = parts[0].strip().upper()
                value = int(parts[1].strip())
                result_dict[ticker] = value
    
    return result_dict

cik_dict = convert_cik_list('./cik.txt')

values = {i:[0]*5 for i in cik_dict.keys()}
missing_op_income_loss = []

for key, val in cik_dict.items():
    try:
        zeros_to_add = 10-len(str(val))
        new_cik = ""
        for i in range(zeros_to_add):
            new_cik+="0"
        final_cik = new_cik + str(val)
        url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{final_cik}.json"
        req = requests.get(url = url, headers= {"User-Agent":"rie@gmail.com", "Accept-Encoding":"gzip, deflate, br", "Host":"data.sec.gov", "Connection":"keep-alive"})
        operating_income_losses = req.json()['facts']['us-gaap']['OperatingIncomeLoss']
        values[key][0], values[key][1], values[key][2], values[key][3], values[key][4] = operating_income_losses[-1], operating_income_losses[-2], operating_income_losses[-3], operating_income_losses[-4], operating_income_losses[-5]
    except Exception as e:
        missing_op_income_loss.append(key)

len(missing_op_income_loss)

12084

In [18]:
import requests
import os
from dotenv import load_dotenv
load_dotenv()

key = os.environ['FMP_KEY']
def get_cashflow(ticker: str, span:int = 5) -> str:
    url = f'https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}?period=annual&apikey={key}&limit={span}'
    response = requests.get(url)
    return response.json()

def get_five_year_fcf(ticker: str) -> list[str]:
    return [i['freeCashFlow'] for i in get_cashflow(ticker)]



def get_profile(ticker: str, span:int = 5) -> str:
    url = f'https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey={key}'
    response = requests.get(url)
    return response.json()

def five_year_yield(ticker: str) -> float:
    return sum(get_five_year_fcf(ticker)) / get_profile(ticker)[0]['mktCap']

five_year_yield('3420.T')

0.7771079846349065